In [224]:
import pandas as pd
import numpy as np
import ipywidgets as ipw
import bql
import bqwidgets as bqw
from bqplot import pyplot as plt
import bqviz
import bqplot as bqp
from ipywidgets import widgets
import plotly.graph_objs as go
import plotly.express as px
import plotly
import datetime

In [225]:
bq = bql.Service()

In [226]:
def get_MN_Muni_YC():
    MN_muni_YC = 'US Minnesota Muni BVAL Yeild Curve Last Bid'
    TENORS = ['3M','6M', '01', '02', '03', '04', '05', '06', '07', '08','09', '10']
    MN_muni_CURVE_MEMBERS = ['BMSTMN{} Index'.format(t) for t in TENORS]
    bql_item = bq.data.px_last()
    bql_universe = MN_muni_CURVE_MEMBERS
    bql_request = bql.Request(bql_universe, {'YTM':bql_item})
    bql_response = bq.execute(bql_request)
    df_YTM = pd.DataFrame(bql_response.single().df()['YTM'])
    df_YTM = df_YTM.reset_index(drop = True)
    return df_YTM

In [227]:
def tax_equiv_YC(taxrate):
    tax_equiv_YC = 'US Minnesota Muni BVAL Tax Equiv Yeild Curve Last Bid'
    MN_Muni_YTM = pd.DataFrame(get_MN_Muni_YC())
    df_YTM = MN_Muni_YTM/(1-taxrate)
    return df_YTM 

In [228]:
def get_AAA_taxable_YC():
    AAA_taxable_YC = 'US Taxable GO AAA Muni BVAL Yield Curve'
    TENORS = ['3M','6M', '01', '02', '03', '04', '05', '06', '07', '08','09', '10']
    AAA_taxable_CURVE_MEMBERS = ['BVTAXG{} BVLI Index'.format(t) for t in TENORS]
    bql_item = bq.data.px_last()
    bql_universe = AAA_taxable_CURVE_MEMBERS
    bql_request = bql.Request(bql_universe, {'YTM':bql_item})
    bql_response = bq.execute(bql_request)
    df_YTM = pd.DataFrame(bql_response.single().df()['YTM'])
    df_YTM = df_YTM.reset_index(drop = True)
    return df_YTM 


In [229]:
def get_AAA_national_exampt():
    AAA_national_exampt = 'US National Tax Exempt BVAL Muni AAA Yield Curve'
    TENORS = ['3M','6M', '1T', '2T', '3T', '4T', '5T', '6T', '7T', '8T','9T', '10T']
    AAA_national_exampt_CURVE_MEMBERS = ['BVMB{} Index'.format(t) for t in TENORS]
    bql_item = bq.data.px_last()
    bql_universe = AAA_national_exampt_CURVE_MEMBERS
    bql_request = bql.Request(bql_universe, {'YTM':bql_item})
    bql_response = bq.execute(bql_request)
    df_YTM = pd.DataFrame(bql_response.single().df()['YTM'])
    df_YTM = df_YTM.reset_index(drop = True)
    return df_YTM

In [230]:
def AAA_tax_exempt_YC(taxrate):
    tax_equiv_YC = 'US Taxable GO AAA Muni BVAL Yield Curve New tax exempt'
    AAA_YTM = pd.DataFrame(get_AAA_national_exampt())
    df_YTM = AAA_YTM/(1 - taxrate)
    return df_YTM

In [231]:
def get_yield(ticker):
    bond_ticker_univ = str(ticker)  
    bql_items = {"YTM":bq.data.yield_(side = 'bid'),
                "MD": bq.data.mty_years_tdy(mode = 'cached')}
    bql_universe = bond_ticker_univ
    bql_request = bql.Request(bql_universe, bql_items)
    bql_response = bq.execute(bql_request)
    df= bql.combined_df(bql_response).reset_index()
    tenor = df["MD"][1]
    YTM = df["YTM"][0]    
    return [tenor,YTM]

In [232]:
def get_name(ticker):
    bql_item = bq.data.long_comp_name()
    bql_universe = str(ticker)
    bql_request = bql.Request(bql_universe, {"name":bql_item})
    bql_response = bq.execute(bql_request)
    name = bql_response.single().df()["name"][0]
    return name

In [233]:
from scipy.interpolate import CubicSpline 
from datetime import date

In [234]:

def bootstrapping(ytm, linear = True):
    mats = np.array([0.25,0.5,1,2,3,4,5,6,7,8,9,10])
    times = np.arange(.25,10 + 0.9/12, 1/12)
    rates = np.array(ytm.T)[0]
    if linear == True:
        yield_full = np.interp(times, mats, rates)
    elif linear == False:
        cs = CubicSpline(mats,rates, bc_type = 'natural')
        yield_full = cs(times)
    return pd.Series(yield_full)

In [235]:
def search_result(btn = None):
    output_1.clear_output()
    ticker = search_box.value
    yield_and_tenor = get_yield(ticker)
    tax_rate = tax_box_1.value
    name = get_name(ticker)
    tenor = yield_and_tenor[0]
    ytm =  yield_and_tenor[1]/(1 - tax_rate)
    trace_1.add_trace(go.Scatter(x=[tenor],
        y=[ytm],
        mode="markers",
        name = ticker+" "+f"({name})"
        ))
    with output_1:

        trace_1.show()
        trace_1.data = trace_1.data[:-1]        
        trace_2.show()


In [237]:

def model2 (btn = None):
    global trace_1, trace_2
    output_1.clear_output()
    times = pd.Series(np.arange(.25,10 + 0.9/12, 1/12))
    process_box.value = 'requesting data ...'
    linear = mode_box.value
    tax_mode = tax_box.value
    MN_muni = bootstrapping(get_MN_Muni_YC(), linear)
    MN_muni_grossed = bootstrapping(tax_equiv_YC(0.494), linear)
    taxable_aaa =  bootstrapping(get_AAA_taxable_YC(), linear)
    national_aaa_ex = bootstrapping(get_AAA_national_exampt(), linear)
    national_aaa_grossed = bootstrapping(AAA_tax_exempt_YC(0.414), linear)
    tax_acc = [MN_muni_grossed, taxable_aaa, national_aaa_grossed,times ]
    no_tax_acc = [MN_muni, taxable_aaa, national_aaa_ex,times]
    
    process_box.value = "processing data ..."
    capture_date = datetime.datetime.now().strftime("%b %d, %Y")
    if tax_mode == "Taxable Account":
        tax = pd.concat( [tax_equiv_YC(0.494).T, get_AAA_taxable_YC().T, AAA_tax_exempt_YC(0.414).T])
        result = pd.concat(tax_acc, axis = 1,keys =['MN muni (Grossed up, 49.4% tax)','National Taxable AAA','National Exempt AAA (Grossed up, 41.4% Tax)', 'Years'])
        trace_2 = go.Figure(data = [go.Table(header = dict(values = ["Name", "3 Months", "6 Months", "1 Year", "2 Years", "3 Years", "4 Years", "5 Years", "6 Years", "7 Years", "8 Years", "9 Years", "10 Years"],
                                       font = dict(size = 10), align = "left"),
                         cells = dict(values =[ 
                                      ['MN muni (Grossed up, 49.4% tax)','National Taxable AAA','National Exempt AAA (Grossed up, 41.4% Tax)'], 
                                      [i for i in tax[0]],[i for i in tax[1]],[i for i in tax[2]],[i for i in tax[3]],[i for i in tax[4]], [i for i in tax[5]],
                                      [i for i in tax[6]],[i for i in tax[7]],[i for i in tax[8]],[i for i in tax[9]],[i for i in tax[10]],[i for i in tax[11]]
                         ], align = "left"))])
    elif tax_mode == "Non-Taxable Account":
        non_tax = pd.concat( [get_MN_Muni_YC().T, get_AAA_taxable_YC().T, get_AAA_national_exampt().T])
        result = pd.concat(no_tax_acc, axis = 1,keys =['MN muni','National Taxable AAA','National Exempt AAA', 'Years'])
        trace_2 = go.Figure(data = [go.Table(header = dict(values = ["Name", "3 Months", "6 Months", "1 Year", "2 Years", "3 Years", "4 Years", "5 Years", "6 Years", "7 Years", "8 Years", "9 Years", "10 Years"],
                                       font = dict(size = 10), align = "left"),
                         cells = dict(values =[ 
                                      ['MN muni','National Taxable AAA','National Exempt AAA'], 
                                      [i for i in non_tax[0]],[i for i in non_tax[1]],[i for i in non_tax[2]],[i for i in non_tax[3]],[i for i in non_tax[4]], [i for i in non_tax[5]],
                                      [i for i in non_tax[6]],[i for i in non_tax[7]],[i for i in non_tax[8]],[i for i in non_tax[9]],[i for i in non_tax[10]],[i for i in non_tax[11]]
                         ], align = "left"))])
    result.set_index("Years", inplace = True)
    trace_1 = px.line(result, x = result.index, y = result.columns, title = "Yield Curves", labels = {"value" : "Yield", "variable" : "Index & CUSIP","Years" : f"Years<br><sup>Capture in {capture_date}</sup>"})
    process_box.value = 'done'
    with output_1:
        trace_1.show()
        trace_2.show()


In [238]:
linear_string = ipw.Label() 
linear_string.value = "Bootstrapping method" 
true_false = [("Linear",True), ("CubicSpline",False)]
mode_box = ipw.Dropdown(options=true_false,layout={'width':'190px'})
tax_status = ["Taxable Account","Non-Taxable Account"]
tax_box = ipw.Dropdown(options=tax_status,layout={'width':'190px'}, description = "Account")
button1 = ipw.Button(description = 'run', color = "o")
process_box = ipw.Label()
button1.on_click(model2)
control_1= ipw.HBox([linear_string, mode_box,tax_box, button1, process_box])
output_1 = ipw.Output()
string_des = ipw.Label()
string_des.value = "Please enter the CUSIP for bond here"
search_box = bqw.TickerAutoComplete()
tax_option = [("No tax benefit",0), ('Federal tax exempt',0.414), ('Federal and State tax exempt',0.494)]
tax_box_1 = ipw.Dropdown(options=tax_option,layout={'width':'300px'}, description = "Tax Option")
button2 = ipw.Button(description = "Search bond")
button2.on_click(search_result)
search_1 = ipw.HBox([string_des, search_box,tax_box_1,button2])
display(control_1, search_1, output_1)

Output()